#### Copie de Mehdi - ne pas enregistrer dessus (génération de conflits)

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
from glob import glob
import pickle

import numpy as np
import pandas as pd

import nltk
import re
import codecs
import unidecode
#pip install unidecode
import mpld3
# pip install mpld3
import stop_words
# pip install stop-words
from nltk import SnowballStemmer, pos_tag, word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import HashingVectorizer,TfidfTransformer,TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import *
from sklearn.linear_model import LogisticRegression
from sklearn.svm import *
from sklearn.semi_supervised import *

** Lecture des données **

In [2]:
class CV:
    def __init__(self, filename, title, cv, doc_type):
        self.filename = filename
        self.title = title
        self.cv = cv
        self.doc_type = doc_type

In [3]:
def delete_context_part(ao):
    """Supprime la partie de l'appel d'offre consacrée au contexte pour ceux qui suivent le pattern SNCF"""
    split_ao = ao.split('DESCRIPTION DE LA MISSION')
    if len(split_ao)>1:
        ao_light = split_ao[1] 
    else :
        ao_light = split_ao[0]
    return ao_light

In [4]:
def delete_contact_part(ao):
    """Supprime la partie de l'appel d'offre consacrée au contact et adresse pour 
    ceux suivant le même pattern"""
    split_ao = ao.split('CONDITIONS D’EXECUTION')
    if len(split_ao)>1:
        split_ao_2 = split_ao[1].split('AUTRES POINTS')
        if len(split_ao_2)>1:
            ao_light = split_ao[0] + " " + split_ao_2[1]
        else :
            ao_light = split_ao[0]
    else :
        ao_light = split_ao[0]
    return ao_light

In [5]:
#download cv in a list
def load_Doc_list(nombre, path, doc_type):
    liste_paths = [path+directory for directory in os.listdir(path)]
    #print(liste_paths)
    liste_cv = []
    dico_cv = {}
    for path in liste_paths :
        cv = CV(0, 0, 0, 0)
        cv.filename = os.path.basename(path)
        
        try:
            if doc_type == 'ao':
                cv.cv = delete_contact_part(open(path).read())
                cv.cv = delete_context_part(cv.cv)
            else :
                cv.cv = open(path).read()
            cv.doc_type = doc_type
            liste_cv.append(cv.cv)
            dico_cv[cv.filename.split(".")[0]] = cv#.
        except UnicodeDecodeError:
            print("Error !")
            print(path)
    return liste_cv, dico_cv

In [6]:
print("Loading titles")
def load_title_list(nombre, path):
    path = "../Maha/data/"
    dico_titles = {}
    filenames = sorted(glob(os.path.join(path,"*.csv")))
    for file in filenames[:nombre]:
        with open(file, encoding='latin-1') as csvfile:
            readCSV = csv.reader(csvfile, delimiter=';')
            for row in readCSV:
                dico_titles[row[0]] = row[1]
    return dico_titles

Loading titles


In [7]:
path_AO = '../Commun/Data_Talan/txt_ao/'
path_CV = '../Commun/Data_Talan/txt/'

In [8]:
liste_cv, dico_cv = load_Doc_list(200, path_CV, 'cv')
liste_AO, dico_AO = load_Doc_list(200, path_AO, 'ao')

# ne fonctionne pas, pas grave
#dico_CV_titles = load_title_list(150, path_CV)
#dico_AO_titles = load_title_list(150, path_AO)

# on concatène tous les éléments dans une même liste :
#liste_cv_indeed = liste_cv_indeed0 + liste_AO_indeed
dico_global = dict(dico_cv)
dico_global.update(dico_AO)


In [9]:
dico_global

{'2015 Consultation conduite projet et accompagnement projet_V2': <__main__.CV at 0x1192738d0>,
 '2015 Consultation-GCL-CMDB_Bornéo_V2': <__main__.CV at 0x119273908>,
 '245_Description_mission_CDPT_3P2Iv2_Consultation_n°245': <__main__.CV at 0x119273860>,
 '246_DSIRI_-_IVRC_-_MOE_et_intégrateur_BROKER_IV_Consultation_n°246': <__main__.CV at 0x119273550>,
 '256_ISIDA_1810_CONSULTANT_FONCTIONNEL_ERP_CONFIRME_MAXIMO_MOBILITE_Consultation n°256': <__main__.CV at 0x1192736d8>,
 '256_ISIDA_1810_CONSULTANT_FONCTIONNEL_ERP_CONFIRME_MAXIMO_MOBILITE_Consultation_n°256': <__main__.CV at 0x119273940>,
 '259_ISIDA_1816_Consultation_Mission_ASTR_QMAQMSI_Décisionnel_Consultation n°259': <__main__.CV at 0x1192736a0>,
 '259_ISIDA_1816_Consultation_Mission_ASTR_QMAQMSI_Décisionnel_Consultation_n°259': <__main__.CV at 0x119273898>,
 '262_ISIDA_1813_Consultation_Mission_ASTR_MOA_Consultant_fonct_erp_junior_Maximo_Consultation_n°262': <__main__.CV at 0x119277198>,
 '304_EB SAS-24-02-2016': <__main__.CV at

In [10]:
len(dico_global)

691

### Cleaning fonctions

#### Suppression des sauts de ligne

In [11]:
import string, re

In [12]:
regex = re.compile('[%s]' % '(\\n)*(\\x0c)*')
def del_line_feed(s):  
    """Delete \n in the text"""
    return regex.sub(' ', s)

In [13]:
for key, cv in dico_global.items():
    cv.cv = del_line_feed(cv.cv).lower()
    dico_global[key] = cv

In [14]:
for key, value in dico_global.items():
    print(value.cv)
    break

consultant junior amoa  formation  1 janvier 2011 - 1 novembre 2011 master 2 entrepreneuriat: stratégie marketing et commerciale, elaboration du business plan, gestion financière sup'career, paris  1 octobre 2007 - 1 décembre 2008 master 1 entrepreneuriat: conception et evaluation de projets, management des organisations, contrôle de gestion  université paris 12,  créteil  compétences  savoir-faire: gestion de la relation client - conseil - gestion de portefeuille de clients professionnels - techniques de planification - gestion de projets - marketing informatique logiciels de gestion: erp: crm  bureautique: pack office  word, excel, access, powerpoint   expérience professionnelle  27 novembre 2015 - 29 septembre 2016 téléconseillère, qualigaz, aubervilliers 8 septembre 2014 - 9 mai 2015 conseillère clientèle, comearth, orsay 11 novembre 2013 - 29 août 2014 assistante maternelle, particulier, paris 1 février 2012 - 15 mai 2012 conseillère clientèle, olympus france, rungis 1 décembre 20

#### Suppression ponctuation

In [15]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [16]:
#le maintient de la ponctuation pertube le stop words, apostrophe gérée dans text_treatment
regex = re.compile('[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_{|}~')) 

def del_punct(s):  
    """Delete punctuation in the text"""
    return regex.sub(' ', s)

In [17]:
#test 
#liste_cv_no_punc = [del_punct(text) for text in liste_cv]
for key, cv in dico_global.items():
    cv.cv = del_punct(cv.cv)
    dico_global[key] = cv

In [18]:
#liste_cv_no_punc[0]
for key, value in dico_global.items():
    print(value.cv)
    break

consultant junior amoa  formation  1 janvier 2011   1 novembre 2011 master 2 entrepreneuriat  stratégie marketing et commerciale  elaboration du business plan  gestion financière sup'career  paris  1 octobre 2007   1 décembre 2008 master 1 entrepreneuriat  conception et evaluation de projets  management des organisations  contrôle de gestion  université paris 12   créteil  compétences  savoir faire  gestion de la relation client   conseil   gestion de portefeuille de clients professionnels   techniques de planification   gestion de projets   marketing informatique logiciels de gestion  erp  crm  bureautique  pack office  word  excel  access  powerpoint   expérience professionnelle  27 novembre 2015   29 septembre 2016 téléconseillère  qualigaz  aubervilliers 8 septembre 2014   9 mai 2015 conseillère clientèle  comearth  orsay 11 novembre 2013   29 août 2014 assistante maternelle  particulier  paris 1 février 2012   15 mai 2012 conseillère clientèle  olympus france  rungis 1 décembre 20

** Reconnaissance du langage du CV**

In [19]:
def _calculate_languages_ratios(text):
    """
    Calculate probability of given text to be written in several languages and
    return a dictionary that looks like {'french': 2, 'spanish': 4, 'english': 0}
    """

    languages_ratios = {}

    '''
    nltk.wordpunct_tokenize() splits all punctuations into separate tokens
    
    >>> wordpunct_tokenize("That's thirty minutes away. I'll be there in ten.")
    ['That', "'", 's', 'thirty', 'minutes', 'away', '.', 'I', "'", 'll', 'be', 'there', 'in', 'ten', '.']
    '''

    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens] #from text get list of word in minuscule

    
    for language in stopwords.fileids(): # pour chaque langue
        stopwords_set = set(stopwords.words(language)) #je mets les stop words du langage dans un set
        words_set = set(words) #je mets les mots de mon texte dans un set
        #je prends l'intersection entre les mots de mon texte et les mots du stopwords dans le langage donné
        common_elements = words_set & stopwords_set
        
        #je compute mon score comme le nombre d'éléments en communs dictionnaire [langage : score]
        languages_ratios[language] = len(common_elements) # language "score"

    return languages_ratios

In [20]:
import nltk
nltk.download('stopwords')
stopwords.fileids()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mehdiregina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish',
 'turkish']

In [21]:
def get_cv_langue(liste_cv, language) :
    """Return resume witten in the specified language in parameter"""
    liste_2 = []
    for cv in liste_cv:
        if max(_calculate_languages_ratios(cv),key =_calculate_languages_ratios(cv).get)=='french':
            liste_2.append(cv)
    return liste_2

In [22]:
for k, cv in dico_global.items():
    calc = _calculate_languages_ratios(cv.cv)
    if max(calc, key=calc.get)!='french':
        ##del dico_cv[key]
        cv.cv = ""
        dico_global[k] = cv

dico_global = {key:dico_global[key] for key in dico_global.keys() if dico_global[key].cv != ""}        
len(dico_global)

668

In [23]:
for key, value in dico_global.items():
    print(value.cv)
    break

consultant junior amoa  formation  1 janvier 2011   1 novembre 2011 master 2 entrepreneuriat  stratégie marketing et commerciale  elaboration du business plan  gestion financière sup'career  paris  1 octobre 2007   1 décembre 2008 master 1 entrepreneuriat  conception et evaluation de projets  management des organisations  contrôle de gestion  université paris 12   créteil  compétences  savoir faire  gestion de la relation client   conseil   gestion de portefeuille de clients professionnels   techniques de planification   gestion de projets   marketing informatique logiciels de gestion  erp  crm  bureautique  pack office  word  excel  access  powerpoint   expérience professionnelle  27 novembre 2015   29 septembre 2016 téléconseillère  qualigaz  aubervilliers 8 septembre 2014   9 mai 2015 conseillère clientèle  comearth  orsay 11 novembre 2013   29 août 2014 assistante maternelle  particulier  paris 1 février 2012   15 mai 2012 conseillère clientèle  olympus france  rungis 1 décembre 20

** Preprocessing du text **

In [24]:
def text_treatment (text):
    text = text.lower()
    text = text.replace("\x00", '').replace("\x01", '').replace("\x02", '').replace("\x03", '') \
    .replace("\x04", '').replace("\x05", '').replace("\x06", '').replace("\x07", '').replace("\x08", '') \
    .replace("\x0e", '').replace("\x11", '').replace("\x12", '').replace("\x10", '').replace("\x19", '') \
    .replace("\x1b", '').replace("\x14", '').replace("\x15", '').replace('/', '').replace('=', '').replace("〓", "") \
    .replace("»", "").replace("«", "").replace("¬", "").replace('`', '').replace (" -", "").replace("•", "")\
    .replace("l'", "").replace("l’", "").replace("l´", "").replace("d’", "").replace("d'", "").replace("d´","")\
    .replace("j’", "").replace("j'", "").replace("j´","").replace("n’", "").replace("n'", "").replace("n´","")\
    .replace("”", "").replace("~", "").replace("§", "").replace("¨", "").replace("©", "").replace("›", "")\
    .replace("₋", "").replace("→", "").replace("⇨", "").replace("∎", "").replace("√", "").replace("□", "")\
    .replace("*", "").replace("&", "").replace("►", "").replace("◊", "").replace("☞", "").replace("#", "")\
    .replace("%", "").replace("❖", "").replace("➠", "").replace("➢", "").replace("", "").replace("✓", "") \
    .replace("√", "").replace("✔", "").replace("♦", "").replace("◦", "").replace("●", "").replace("▫", "")\
    .replace("▪", "").replace("…", "").replace("þ", "").replace("®", "").replace('', '').replace("...", "")\
    .replace(" o ", "")
    text = unidecode.unidecode(text) # remove accent
    return text

In [25]:
#On supprime les caractères étranges et accents
#liste_cv_treated = [text_treatment(text) for text in liste_cv_fr]
for key, cv in dico_global.items():
    cv.cv = text_treatment(cv.cv)
    dico_global[key] = cv

In [26]:
#liste_cv_treated[0]
for key, value in dico_global.items():
    print(value.cv)
    break

consultant junior amoa  formation  1 janvier 2011   1 novembre 2011 master 2 entrepreneuriat  strategie marketing et commerciale  elaboration du business plan  gestion financiere sup'career  paris  1 octobre 2007   1 decembre 2008 master 1 entrepreneuriat  conception et evaluation de projets  management des organisations  controle de gestion  universite paris 12   creteil  competences  savoir faire  gestion de la relation client   conseil   gestion de portefeuille de clients professionnels   techniques de planification   gestion de projets   marketing informatique logiciels de gestion  erp  crm  bureautique  pack office  word  excel  access  powerpoint   experience professionnelle  27 novembre 2015   29 septembre 2016 teleconseillere  qualigaz  aubervilliers 8 septembre 2014   9 mai 2015 conseillere clientele  comearth  orsay 11 novembre 2013   29 aout 2014 assistante maternelle  particulier  paris 1 fevrier 2012   15 mai 2012 conseillere clientele  olympus france  rungis 1 decembre 20

** Gestion des stop words **

In [27]:
#generate stopwords
stop_words_py = set(stop_words.get_stop_words('french'))

# attention certains stop words pourraient être utiles par la suite
stopwords_set_manuel = set(["an", "ans", 'les', 'moins', 'd\'un','janvier', 'fevrier', 'février', 'mars', 'avril', \
                 'mai', 'juin', 'juillet', 'aout', 'août', 'septembre', 'octobre', 'novembre', 'décembre', \
                  'decembre', 'moins', 'mise', 'universit\xc3\xa9', 'université', 'universite', 'ion','sage', \
                  'o', 'rac', 'vers', 'via', 'p\xc3\xa9rim\xc3\xa8tre', 'périmètre','et','paris','x',"\x00",\
                          "\x01","\x02", "\x03","\x04","\x05","\x06","\x07","\x08","\x09","\x0e","\x0e","\x11",\
                           "\x12","\x13","\x14","\x15","\x16","\x17","\x18","\x19","transport","puis","lieu",\
                           "adresse","entre",'dun','dune','chez','boulognebillancourt','bt','etc','recrutement','main',\
                           'and', 'paie','paiement','environ','place','france','paris','mois','mobile','mobiles',\
                           'nanterre','source','sources','concerne','concernant','of','non','notes','rh','minimum',\
                           'maximum','bac','site','sites','actuellement','telephone','telephonique','telephoniques','ca','demenager',\
                           'demenagement','participer','participation','lycee','baccalaureat','lien','liens','in',\
                           'indeed','email','indeedcomrd7e8913ed00d0384','aujourhui','afin','toujours','enterprise',\
                           "guide","10g","11g","9i",'ad','v10','v2','v3','v5','v6','v8','v9','pablo','neruda',\
                           'dec','stelsia','cid1','sens','va','24h24', '7j7', 'levalloisperret','louis', 'armand', 'ermont',\
                            'localisation','anne', 'perot',' s ','mission', 'date', 'debut','fin','avenue', 'val', 'oise',\
                           'echeant','bout','division','prestations', 'intellectuelles', 'contexte','descroix','24h', '7j',\
                           'disposition','places','moyens', 'sncf', 'mis','rue','levallois','perret','scnf','ainsi',\
                           "jusqu'au","jusqu'a",'ast','exercant','souhaitee'])
stop_words_main = stop_words_py | stopwords_set_manuel
stop_words_main = list(stop_words_main)
print("taille stop words liste : ", len(stop_words_main))

taille stop words liste :  416


In [28]:
stop_words_main

['cela',
 'afin',
 'serez',
 'sommes',
 'lycee',
 'v9',
 'sans',
 'neruda',
 'sujet',
 'juin',
 'n',
 'octobre',
 'decembre',
 'début',
 'eux',
 'auras',
 'eussiez',
 'du',
 'nous',
 'ainsi',
 'nommés',
 'levallois',
 'fus',
 'eus',
 'bon',
 'va',
 'bac',
 'de',
 'maximum',
 '7j7',
 'levalloisperret',
 'tout',
 'quelles',
 'contexte',
 'environ',
 'localisation',
 'aurai',
 'nom',
 'enterprise',
 'vous',
 'te',
 'telephone',
 'devriez',
 'seras',
 'mis',
 'vers',
 'personne',
 '\t',
 'aient',
 'ai',
 'devoir',
 'sur',
 'perret',
 'avec',
 'l',
 'étés',
 "jusqu'au",
 'ast',
 'soyez',
 'seront',
 'actuellement',
 'ad',
 'mot',
 'nos',
 'cet',
 '\x12',
 'lien',
 'août',
 'telephoniques',
 'lui',
 'cette',
 'eue',
 'an',
 'dois',
 'pas',
 '10g',
 'étions',
 'ils',
 'dù',
 'fin',
 'faire',
 'me',
 'où',
 'juillet',
 'étiez',
 'se',
 '\x13',
 'anne',
 'x',
 '\x05',
 'un',
 'avenue',
 'seraient',
 'transport',
 ' s ',
 "d'un",
 'droite',
 '\x17',
 'mission',
 'dos',
 'fussions',
 'à',
 'v8',


In [29]:
#voir si utile
def remove_stopwords(text,stopwords_list):
    text_temp = " ".join(text.split())+" "
    for word in stopwords_list:
        text_temp = text_temp.replace(" "+word+" ", " ")
    return text_temp

In [30]:
for key, cv in dico_global.items():
    cv.cv = remove_stopwords(cv.cv, stop_words_main)
    dico_global[key] = cv

In [31]:
for key, value in dico_global.items():
    print(value.cv)
    break

consultant junior amoa formation 1 2011 1 2011 master 2 entrepreneuriat strategie marketing commerciale elaboration business plan gestion financiere sup'career 1 2007 1 2008 master 1 entrepreneuriat conception evaluation projets management organisations controle gestion 12 creteil competences savoir gestion relation client conseil gestion portefeuille clients professionnels techniques planification gestion projets marketing informatique logiciels gestion erp crm bureautique pack office word excel access powerpoint experience professionnelle 27 2015 29 2016 teleconseillere qualigaz aubervilliers 8 2014 9 2015 conseillere clientele comearth orsay 11 2013 29 2014 assistante maternelle particulier 1 2012 15 2012 conseillere clientele olympus rungis 1 2010 30 2011 assistante administrative mtpro services pierrelaye 1 2009 14 2010 chargee assistance automobile axa assistance chatillon 1 2008 30 2008 stage assistant chef projet smartbox langues anglais conversationnel formation encours anglai

In [32]:
#SnowballStemmer 
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french")

In [33]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters 
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters 
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [34]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []

for key, cv in dico_global.items():
    #cv.cv = remove_stopwords(cv.cv, stop_words_main)
    #dico_cv[key] = cv
    allwords_stemmed = tokenize_and_stem(cv.cv) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    allwords_tokenized = tokenize_only(cv.cv)
    totalvocab_tokenized.extend(allwords_tokenized)


#for i in liste_cv_no_stop:
    #allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    #totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    #allwords_tokenized = tokenize_only(i)
    #totalvocab_tokenized.extend(allwords_tokenized)

In [35]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')
len(vocab_frame['words'].unique())

there are 260259 items in vocab_frame


18456

## Import vocabulary list

In [39]:
with open("vocab.txt","r") as file:
    vocab_list = file.read()
vocab_list = vocab_list.split(",")[:-1]

### ACP X KMEANS

In [45]:
#tf_vect = TfidfVectorizer(stop_words=stop_words_main,max_df=0.5,min_df=0.05,\
                           #preprocessor=text_treatment,tokenizer=tokenize_and_stem, ngram_range=(3,3))
    
# sans stemming : 
    
tf_vect = TfidfVectorizer(stop_words=stop_words_main,vocabulary=vocab_list,\
                           preprocessor=text_treatment,tokenizer=tokenize_only)

In [46]:
#matrix = count_vec.fit_transform(liste_cv_no_stop)
liste_cv_no_stop = list()
liste_title = list()
liste_doctype = list()
for key, cv in dico_global.items():
    cv.cv = remove_stopwords(cv.cv, stop_words_main)
    liste_cv_no_stop.append(cv.cv)
    liste_title.append(cv.filename)
    liste_doctype.append(cv.doc_type)

In [47]:
bow_idf_indeed = tf_vect.fit_transform(liste_cv_no_stop)

In [48]:
print(bow_idf_indeed.shape)
bow_idf_indeed
print(len(liste_cv_no_stop))

print(len(liste_cv_no_stop))
print(len(liste_cv_no_stop))

bow_idf_indeed[0] # 1er doc
liste_cv_no_stop[0]
liste_title[0]
set(liste_doctype)

(668, 361)
668
668
668


{'ao', 'cv'}

In [49]:
def compute_eucl_dist(v1, v2):
    a = np.array(v1)
    b = np.array(v2)
    return np.dot((a-b), np.transpose(a-b))

In [50]:
len(liste_cv_no_stop)

668

In [51]:
class AO_CV_Dist:
    def __init__(self, ao, cv, dist):
        self.ao = ao
        self.cv = cv
        self.dist = dist
        
        
    def to_dict(self):
        return {
            'ao': self.ao,
            'cv': self.cv,
            'dist': self.dist,
        }
    
        

### compute euc. distance ao cv

In [52]:
list_ao_cv = list()
list_df = list()
for i in range(0, len(liste_cv_no_stop)):
    if liste_doctype[i] == 'ao':
        list_by_ao = list()
        for j in range(0, len(liste_cv_no_stop)):
            if i > j:
                if liste_doctype[j] == 'cv':
                    local_dist = compute_eucl_dist(bow_idf_indeed[i], bow_idf_indeed[j])[0,0]
                    ao_cv = AO_CV_Dist(liste_title[i], liste_title[j], local_dist)
                    list_ao_cv.append(ao_cv)
                    list_by_ao.append(ao_cv)
        df = pd.DataFrame.from_records([s.to_dict() for s in list_by_ao])
        list_df.append(df.sort_values(by='dist'))           

In [55]:
print(list_df[17].iloc[0:10])

                                                    ao               cv  \
540  245_Description_mission_CDPT_3P2Iv2_Consultati...   CV_5_BROSS.txt   
127  245_Description_mission_CDPT_3P2Iv2_Consultati...  FGICV TALAN.txt   
370  245_Description_mission_CDPT_3P2Iv2_Consultati...    CV_5_DIAG.txt   
533  245_Description_mission_CDPT_3P2Iv2_Consultati...  TJACV TALAN.txt   
60   245_Description_mission_CDPT_3P2Iv2_Consultati...     CV_5_DOT.txt   
304  245_Description_mission_CDPT_3P2Iv2_Consultati...         MARI.txt   
209  245_Description_mission_CDPT_3P2Iv2_Consultati...  PVNCV TALAN.txt   
27   245_Description_mission_CDPT_3P2Iv2_Consultati...     CV_MDM_1.txt   
473  245_Description_mission_CDPT_3P2Iv2_Consultati...     CV_5_NOU.txt   
146  245_Description_mission_CDPT_3P2Iv2_Consultati...     CV_5_JAW.txt   

         dist  
540  1.369347  
127  1.411723  
370  1.428189  
533  1.435095  
60   1.438485  
304  1.468788  
209  1.492266  
27   1.494986  
473  1.501688  
146  1.504013 

In [46]:
for i in range(len(list_df)):
    print("idx:{}, {}".format(i,list_df[i].iloc[0,0]))

idx:0, 343_ISIDA_1948_Prestation CDP_Concepteur_SPOT Outillage_2016.txt
idx:1, 304_EB_SAS-24-02-2016.txt
idx:2, Fiche Expression de Besoin- 909 - Developpeur Tableau Software.txt
idx:3, 246_DSIRI_-_IVRC_-_MOE_et_intégrateur_BROKER_IV_Consultation_n°246.txt
idx:4, Consultation_HCC_AST_3485-_Fiche_expression_de_besoin_-_Chef_de_Projet.txt
idx:5, 388_ISIDA_1993_Prestation ASTR MOA Fonctionnel Maximo OPTISPOT-v0.4_1.txt
idx:6, HCC_AST_288_Datastage_confirmé-_CSI_DEC.txt
idx:7, Consultation - Mission Progiciels-v4.txt
idx:8, Consultation_HCC_AST_3515_-_Gestionnaire_de_patrimoine_DEC_-_Accès_Réseau.txt
idx:9, 259_ISIDA_1816_Consultation_Mission_ASTR_QMAQMSI_Décisionnel_Consultation n°259.txt
idx:10, 256_ISIDA_1810_CONSULTANT_FONCTIONNEL_ERP_CONFIRME_MAXIMO_MOBILITE_Consultation n°256.txt
idx:11, 481_Renouvellement_cd_STELSIA_32524-0000012774.txt
idx:12, 390_Cahier des charges _Transilien_chargé d_étude stat.txt
idx:13, Consultation_Portail__Stocks_ELAN_15-10-2015.txt
idx:14, Consultation

--------------

In [46]:
matrix = bow_idf_indeed
matrix.shape

(271, 1029)

In [47]:
dist = 1 - cosine_similarity(matrix)
dist

array([[  0.00000000e+00,   8.93059365e-01,   9.58214626e-01, ...,
          8.90518063e-01,   9.24013152e-01,   1.00000000e+00],
       [  8.93059365e-01,  -4.44089210e-16,   9.47156268e-01, ...,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  9.58214626e-01,   9.47156268e-01,  -2.22044605e-16, ...,
          9.61752124e-01,   9.70149385e-01,   1.00000000e+00],
       ..., 
       [  8.90518063e-01,   1.00000000e+00,   9.61752124e-01, ...,
          0.00000000e+00,   8.92587903e-02,   1.00000000e+00],
       [  9.24013152e-01,   1.00000000e+00,   9.70149385e-01, ...,
          8.92587903e-02,   1.11022302e-16,   9.77536041e-01],
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00, ...,
          1.00000000e+00,   9.77536041e-01,   0.00000000e+00]])

In [48]:
import scipy.spatial.distance as ssd
from scipy import cluster
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

# convert the redundant n*n square matrix form into a condensed nC2 array
dist2=np.array(dist)
dist2[dist2 < 0] = 0
np.fill_diagonal(dist2, 0)
X = ssd.squareform(dist2) # distArray[{n choose 2}-{n-i choose 2} + (j-i-1)] is the distance between points i and j
#Z = cluster.hierarchy.linkage(X, method='ward', metric='euclidean')

In [49]:
X.shape

(36585,)

** Suppression des doublons **

In [50]:
#Ajout une étape pour supprimer les doublons
buffer = pd.DataFrame(data=bow_idf_indeed.toarray())
buffer.drop_duplicates(inplace=True)
bow_idf_indeed = buffer.values
bow_idf_indeed = shuffle(bow_idf_indeed)

** Clustering **

In [79]:
num_clusters = 6

km = KMeans(n_clusters=num_clusters, n_jobs=-1)

%time km.fit(matrix)

clusters = km.labels_.tolist()
#clusters

CPU times: user 80.6 ms, sys: 35.6 ms, total: 116 ms
Wall time: 1.03 s


In [80]:
# Here, I create a dictionary of titles, cv body, the cluster assignment
cv_titles = [cv.filename for cv in dico_global.values()]
cv_bodies = [cv.cv for cv in dico_global.values()]
cv_types = [cv.doc_type for cv in dico_global.values()]

cvs = { 'title': cv_titles, 'cv': cv_bodies, 'cluster': clusters, 'doc_type':cv_types }
#cvs = { 'title': cv_titles,  'cluster': clusters }

In [81]:
print(len(cv_titles))
print(len(cv_bodies))
print(len(clusters))

271
271
271


In [82]:
frame = pd.DataFrame(cvs, index = [clusters] , columns = ['title', 'cluster', 'doc_type'])

In [83]:
frame.head()

,title,cluster,doc_type
2,ACH.txt,2,cv
4,ADD.txt,4,cv
1,ADECV TALAN.txt,1,cv
1,AGNCV TALAN.txt,1,cv
4,AIT.txt,4,cv


In [84]:
frame['cluster'].value_counts() #number of cv per cluster

4    84
2    82
1    35
0    30
5    23
3    17
Name: cluster, dtype: int64

In [85]:
frame[frame ['cluster']==0]

,title,cluster,doc_type
0,CV_BD_2.txt,0,cv
0,CV_DS_1.txt,0,cv
0,CV_DS_10.txt,0,cv
0,CV_DS_11.txt,0,cv
0,CV_DS_12.txt,0,cv
0,CV_DS_15.txt,0,cv
0,CV_DS_2.txt,0,cv
0,CV_DS_4.txt,0,cv
0,CV_DS_5.txt,0,cv
0,CV_DS_6.txt,0,cv


In [86]:
frame[frame ['cluster']==1]

,title,cluster,doc_type
1,ADECV TALAN.txt,1,cv
1,AGNCV TALAN.txt,1,cv
1,AKACV TALAN.txt,1,cv
1,APHCV TALAN.txt,1,cv
1,APICV TALAN.txt,1,cv
1,BLACV TALAN.txt,1,cv
1,BPUCV TALAN.txt,1,cv
1,CAPCV TALAN.txt,1,cv
1,CLSCV TALAN.txt,1,cv
1,ELCCV TALAN.txt,1,cv


In [87]:
frame[frame ['cluster']==2]

,title,cluster,doc_type
2,ACH.txt,2,cv
2,BAR.txt,2,cv
2,BENA.txt,2,cv
2,BGHCV TALAN.txt,2,cv
2,BKICV TALAN.txt,2,cv
2,BLPCV TALAN.txt,2,cv
2,BOA.txt,2,cv
2,BOUT.txt,2,cv
2,BUDU.txt,2,cv
2,CHEK.txt,2,cv


In [88]:
frame[frame ['cluster']==3]

,title,cluster,doc_type
3,AO_Consultation_Build_-_909_-_Tableau_Software...,3,ao
3,CC_AST_283_-_Consultation_Build_-_904PAR_-_Che...,3,ao
3,CC_AST_290_-_909_PAR_-_Concepteur_-_développe...,3,ao
3,CC_AST_3466_-_912_PAR_-_Chef_de_Projet_Big_Dat...,3,ao
3,CC_AST_3489_-_Besoin_Socle_-_912_PAR_-_Chef_de...,3,ao
3,Consultation_HCC_AST_3467-_Chef_de_projet_Déc...,3,ao
3,Consultation_HCC_AST_3484_-_Fiche_expression_d...,3,ao
3,Consultation_HCC_AST_3485-_Fiche_expression_de...,3,ao
3,Consultation_HCC_AST_3486_-_Chef_de_Projet-_Be...,3,ao
3,Consultation_HCC_AST_3496_-_Consultant_confirm...,3,ao


In [89]:
frame[frame ['cluster']==4]

,title,cluster,doc_type
4,ADD.txt,4,cv
4,AIT.txt,4,cv
4,AME.txt,4,cv
4,ANDA.txt,4,cv
4,BOUD.txt,4,cv
4,CV - TalanSolutions - AGU.txt,4,cv
4,CV - TalanSolutions - CJD.txt,4,cv
4,CV - TalanSolutions - DPE.txt,4,cv
4,CV _TalanSolutions_AAG AOUT 2017.txt,4,cv
4,CV AAU_TS2017.txt,4,cv


In [90]:
frame[frame ['cluster']==5]

,title,cluster,doc_type
5,245_Description_mission_CDPT_3P2Iv2_Consultati...,5,ao
5,246_DSIRI_-_IVRC_-_MOE_et_intégrateur_BROKER_...,5,ao
5,256_ISIDA_1810_CONSULTANT_FONCTIONNEL_ERP_CONF...,5,ao
5,262_ISIDA_1813_Consultation_Mission_ASTR_MOA_C...,5,ao
5,327_ISIDA_1918_Prestation ASTR MOA Fonctionnel...,5,ao
5,341_ISIDA_1940_EMI_SPOT_Outillage_Recrutement-...,5,ao
5,343_ISIDA_1948_Prestation CDP_Concepteur_SPOT ...,5,ao
5,388_ISIDA_1993_Prestation ASTR MOA Fonctionnel...,5,ao
5,390_Cahier des charges _Transilien_chargé d_e...,5,ao
5,481_Renouvellement cd STELSIA 32524-0000012774...,5,ao


In [59]:
from __future__ import print_function
cluster_names = {}

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    txt = ""
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        txt = txt + vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0] + ", "
        print(' %s' % vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    cluster_names[i] = txt
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print('     %s;' % title, end='')
        #print('\n')
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words: b'big',

KeyError: "None of [['data', 'science']] are in the [index]"

In [ ]:
import os  # for os.path.basename

import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS

MDS()

# convert two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1, n_jobs=-1)

pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]
print()
print()

In [ ]:
#set up colors per clusters using a dict
cluster_colors = {0: '#1b9e77', 1: '#d95f02', 2: '#7570b3', 3: '#e7298a', 4: '#66a61e'}

#set up cluster names using a dict
#cluster_names = {0: 'informatiques, donnee, gestion, formation, developpement, projet', 
#                 1: 'Police, killed, murders', 
#                 2: 'Father, New York, brothers', 
#                 3: 'Dance, singing, love', 
#                 4: 'Killed, soldiers, captain'}

In [ ]:
cluster_names

In [ ]:
#some ipython magic to show the matplotlib plots inline
%matplotlib inline 

#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters))#, title=cv_titles)) 

#group by cluster
groups = df.groupby('label')


# set up plot
fig, ax = plt.subplots(figsize=(17, 9)) # set size
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, 
            label=cluster_names[name], color=cluster_colors[name], 
            mec='none')
    ax.set_aspect('auto')
    ax.tick_params(\
        axis= 'x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off')
    ax.tick_params(\
        axis= 'y',         # changes apply to the y-axis
        which='both',      # both major and minor ticks are affected
        left='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelleft='off')
    
ax.legend(numpoints=1)  #show legend with only 1 point

##add label in x,y position with the label as the film title
#for i in range(len(df)):
#    ax.text(df.ix[i]['x'], df.ix[i]['y'], df.ix[i]['title'], size=8)  

    
    
plt.show() #show the plot

#uncomment the below to save the plot if need be
#plt.savefig('clusters_small_noaxes.png', dpi=200)


In [ ]:
plt.close()

In [ ]:
#create data frame that has the result of the MDS plus the cluster numbers and titles
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=cv_titles)) 

#group by cluster
groups = df.groupby('label')

#define custom css to format the font and to remove the axis labeling
css = """
text.mpld3-text, div.mpld3-tooltip {
  font-family:Arial, Helvetica, sans-serif;
}

g.mpld3-xaxis, g.mpld3-yaxis {
display: none; }

svg.mpld3-figure {
margin-left: -50px;}
"""

# Plot 
fig, ax = plt.subplots(figsize=(14,6)) #set plot size
#ax.margins(0.03) # Optional, just adds 5% padding to the autoscaling

#iterate through groups to layer the plot
#note that I use the cluster_name and cluster_color dicts with the 'name' lookup to return the appropriate color/label
for name, group in groups:
    points = ax.plot(group.x, group.y, marker='o', linestyle='', ms=18, 
                     label=cluster_names[name], mec='none', 
                     color=cluster_colors[name])
    ax.set_aspect('auto')
    labels = [i for i in group.title]
    
    #set tooltip using points, labels and the already defined 'css'
    tooltip = mpld3.plugins.PointHTMLTooltip(points[0], labels,
                                       voffset=10, hoffset=10, css=css)
    #connect tooltip to fig
    mpld3.plugins.connect(fig, tooltip, TopToolbar())    
    
    #set tick marks as blank
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    
    #set axis as blank
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

    
ax.legend(numpoints=1) #show legend with only one dot

mpld3.display() #show the plot

#uncomment the below to export to html
#html = mpld3.fig_to_html(fig)
#print(html)

In [ ]:
tf_vect.vocabulary_
vocab_liste_pca_indeed

Trop de mots !!

In [ ]:
#cluster centers visualization
def get_kmeans_cluster_words_lsa(bow_idf,lsa_number,cluster_number,word_number,vocab_frame,vocab_liste):
    
    #implement LSA
    svd = TruncatedSVD(lsa_number)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)
    bow_idf_reduced_lsa = lsa.fit_transform(bow_idf)
    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))
    
    #clustering using kmeans
    n_class = cluster_number
    km = KMeans(n_clusters=n_class, init='k-means++', max_iter=100, n_init=1)
    km.fit(bow_idf_reduced_lsa)

    #cluster centroid lsa
    cluster_centroids_lsa = km.cluster_centers_
    liste_cluster_word=[]
    
    #via l'opération inveserse je rebascule les centroids dans l'espace des mots 
    original_space_centroids = svd.inverse_transform(cluster_centroids_lsa)
    #ordre decroissant pour chaque cluster les indices des mots à plus forte corrélation
    idx_ordered_centroids = np.argsort(original_space_centroids,axis=1)[:,::-1] 
    
    for i in range(0, cluster_centroids_lsa.shape[0]): #nombre de clusters
        text = list()
        for j in range(0, word_number): #nombre de mots
            mylist = vocab_liste[idx_ordered_centroids[i,j]].split() # on fait ça car on n utilise pas forcément des unigrammes
            #print(mylist)
            # si stem : 
            #text.append([vocab_frame.loc[el].values.tolist()[0][0] for el in mylist])
            # si pas stem : 
            text.append(mylist)
        print((text))
        liste_cluster_word.append(text)
        
    return bow_idf_reduced_lsa, liste_cluster_word, km.labels_, cluster_centroids_lsa

Possibilité de rebasculer dans l'espace des mots
Puis on ordonne de manière décroissante les vecteurs de coefficients pour chaque cluster
On récupère les indices des mots aux plus grands coefficients pour chaque cluster
Via l'attribut vocabulaire du tf_idf vec on affiche les mots ayant les plus grand coeff pour chaque cluster

In [ ]:
bow_idf_reduced_lsa_indeed, liste_cluster_word_lsa_indeed, labels_lsa_indeed, cluster_centroids_lsa_indeed = \
get_kmeans_cluster_words_lsa(bow_idf_indeed,150,5,6,vocab_frame_indeed,vocab_liste_pca_indeed)

### NMF X KMEANS

In [ ]:
# on part d'un bag of words tf-idf
tf_vect_2 = TfidfVectorizer(stop_words=stop_words_main,max_df=0.8,min_df=0.05,\
                           preprocessor=text_treatment,tokenizer=tokenize_and_stem)

bow_idf_2 = tf_vect_2.fit_transform(liste_cv_indeed_no_stop)

vocab_liste_nmf_indeed = tf_vect_2.get_feature_names()

In [60]:
# cluster centers visualization
def get_kmens_cluster_words_nmf(bow_idf,nmf_number,cluster_number,word_number,vocab_frame,vocab_liste):
    #implement NMF
    print("Fitting the NMF model on with n_samples = "+str(bow_idf.shape[0])+ " and n_features = "+str(bow_idf.shape[1]))
    nmf = NMF(n_components=nmf_number).fit(bow_idf) 
    
    liste_topic = []
    for topic_idx, topic in enumerate(nmf.components_):
        #explain each topix with the words with the strongest coeff
        liste_topic.append(" ".join([vocab_frame.loc[vocab_liste[i]].values.tolist()[0][0]  for i in topic.argsort()[:-10 - 1:-1]]))
    
    #get text matrix in nmf topic space
    bow_idf_reduced_nmf = nmf.fit_transform(bow_idf)
    bow_idf_reduced_nmf_normalized = l2_norm.fit_transform(bow_idf_reduced_nmf) #l2 observation normalization
    
    #clustering using kmeans
    n_class = cluster_number
    km = KMeans(n_clusters=n_class, init='k-means++', max_iter=100, n_init=1)
    km.fit(bow_idf_reduced_nmf_normalized)

    #cluster centroid nmf
    cluster_centroids_nmf = km.cluster_centers_
    
    liste_cluster_word=[]
    
    #via l'opération inveserse je rebascule les centroids dans l'espace des mots !!!
    original_space_centroids = nmf.inverse_transform(cluster_centroids_nmf)
    #ordre decroissant pour chaque cluster les indices des mots à plus forte corrélation
    idx_ordered_centroids = np.argsort(original_space_centroids,axis=1)[:,::-1] 
    
    for i in range(0,cluster_centroids_nmf.shape[0]): #nombre de clusters
        text=""
        for j in range(0,word_number): #nombre de mots
            text+= vocab_frame.loc[vocab_liste[idx_ordered_centroids[i,j]]].values.tolist()[0][0]+ " "
        liste_cluster_word.append(text)
        
    return bow_idf_reduced_nmf_normalized, liste_topic, liste_cluster_word, km.labels_, cluster_centroids_nmf

In [61]:
bow_idf_reduced_nmf_indeed, liste_topic, liste_cluster_nmf_indeed, labels_nmf_indeed, cluster_centroid_nmf_indeed = get_kmens_cluster_words_nmf(bow_idf_2,150,5,6,vocab_frame_indeed,vocab_liste_nmf_indeed)



NameError: name 'bow_idf_2' is not defined

In [62]:
liste_topic

NameError: name 'liste_topic' is not defined

In [63]:
liste_cluster_nmf_indeed

NameError: name 'liste_cluster_nmf_indeed' is not defined

### INDEED SEMI-SUPERVISED AUTOMATIC LABELLING

In [64]:
#get observations cluster distrib
X_texts = [liste_cv_indeed_no_stop[i] for i in buffer.index] #all textes sans doublons
X_idf_reduced = bow_idf_reduced_lsa_indeed
y_cluster_pca = labels_lsa_indeed #labels correspondants

NameError: name 'liste_cv_indeed_no_stop' is not defined

Let's check label distribution

In [ ]:
dict_res = dict()
for i in np.unique(y_cluster_pca):
    dict_res["label_"+str(i)] = len(np.where(y_cluster_pca == i)[0])
dict_res

Idéé : Prenons dans un premiers temps les x observations les plus proche du cluster centroid au sens de la cosine similarity et attribuons leur le label du cluster...

In [ ]:
def get_idx_closest_points(nb_points,X,labels,cluster_centroids):
    distance_idx = np.zeros((len(np.unique(labels)),nb_points))
    for i in np.unique(labels):
        buffer_2 = euclidean_distances(X,np.transpose(cluster_centroids[i].reshape(-1,1)))
        distance_idx[i] = np.argsort(buffer_2,axis=0)[:nb_points].reshape(-1,)
    return distance_idx.astype(int)

In [ ]:
idx_per_cluster = get_idx_closest_points(12,X_idf_reduced,y_cluster_pca,cluster_centroids_lsa_indeed)

In [ ]:
idx_per_cluster

In [ ]:
y_cluster_pca[1002]

Let's build first train set with these labelled data from the clustering + lsa

In [ ]:
def get_input_semi_supervised_draft(idx,X,labels):
    """Method for the manual implementation of semi supervised learning, without sklearn"""
    X_train = np.zeros((idx.shape[0]*idx.shape[1],X.shape[1]))
    y_train = np.zeros(idx.shape[0]*idx.shape[1])
    nb_points = idx.shape[1]
    #set train set
    for i in range(idx.shape[0]): #pour chaque cluster/label
        X_train[i*nb_points:(i+1)*nb_points,:] = X[idx[i]]
        if(i>0):
            y_train[i*nb_points:(i+1)*nb_points] = i
    #set test 
    X_test = np.delete(X,idx.ravel(),axis=0)
    y_test = np.delete(labels,idx.ravel(),axis=0)
    return X_train,y_train,X_test,y_test

In [ ]:
X_train,y_train,X_test,y_test = get_input_semi_supervised_draft(idx_per_cluster,X_idf_reduced,y_cluster_pca)

In [ ]:
X_test.shape

Implementation

    - regularized logistic regression with 1 leave out cross val ? (using probabilities to decide)
    - svm with 1 leave out cross val ? (using distance to hyperplan to decide)
    - tree based methods ? (using class probabilities to decide)

TEST Regression Logistique

In [ ]:
log_reg = LogisticRegression(penalty='l2',C=2)
res = np.zeros((X_train.shape[0]+X_test.shape[0],2)) -1

for i in range(0,150):
    log_reg.fit(X_train,y_train.reshape(-1,))
    pred_prob_max = np.max(log_reg.predict_proba(X_test),axis=1) 
    buffer_pred = log_reg.predict(X_test)
    idx = np.where(pred_prob_max>0.65)
    
    #store results
    print(y_test[idx])
    res[idx,0] = y_test[idx]
    res[idx,1] = buffer_pred[idx]
    
    #update datasets
    X_train = np.vstack((X_train,X_test[idx]))
    y_train = np.vstack((y_train.reshape(-1,1),buffer_pred[idx].reshape(-1,1)))
    X_test = np.delete(X_test,idx,axis=0)
    
    if len(y_test)<10:
        break

In [ ]:
test = res[res[:,0]!=-1]
np.mean(test[:,0]==test[:,1])

In [ ]:
np.unique(res)

Résultat peu concluant, à peine 25% de labéllisation commune avec le clustering

TEST SVM

Rappel augmenter C : risque d'overfitter, on priorise la minimisation de l'erreur sur la maximisation de la marge

In [ ]:
svm = SVC(decision_function_shape='ovr', probability=True,C=10)
res = np.zeros((X_train.shape[0]+X_test.shape[0],2)) -1

for i in range(0,150):
    svm.fit(X_train,y_train.reshape(-1,))
    pred_prob_max = np.max(svm.predict_proba(X_test),axis=1) 
    buffer_pred = log_reg.predict(X_test)
    idx = np.where(pred_prob_max>0.6)
    
    #store results
    print(y_test[idx])
    res[idx,0] = y_test[idx]
    res[idx,1] = buffer_pred[idx]
    
    #update datasets
    X_train = np.vstack((X_train,X_test[idx]))
    y_train = np.vstack((y_train.reshape(-1,1),buffer_pred[idx].reshape(-1,1)))
    X_test = np.delete(X_test,idx,axis=0)
    
    if len(y_test)<10:
        break

In [ ]:
res

In [ ]:
test = res[res[:,0]!=-1]
np.mean(test[:,0]==test[:,1])

SEMI SUPERVISED WITH SEMI SUPERVISED METHODS

In [ ]:
def get_input_semi_supervised(idx,labels):
    """Method to preprocess the data before using sklearn semi-supervised
    return labels array with -1 for unknown labels"""
    y_semi_sup = -1*np.ones(labels.shape[0])
    for label in range(idx.shape[0]):
        print(label)
        y_semi_sup[idx[label]] = label
    return y_semi_sup.astype(int)

In [ ]:
y_semi_sup = get_input_semi_supervised(idx_per_cluster,y_cluster_pca)

In [ ]:
np.where(y_semi_sup==0)

In [ ]:
label_spread = LabelSpreading()

label_spread.fit(X_idf_reduced,y_semi_sup)
test = label_spread.predict(X_idf_reduced)


In [ ]:
test[5]

In [ ]:
y_cluster_pca[5]

In [ ]:
def compare_to_clustering(labels_clustering,new_labels):
    return np.mean(labels_clustering==new_labels)

In [ ]:
compare_to_clustering(test,y_cluster_pca)

#### CV TALAN

In [ ]:
#download talan CVs 
liste_cv_talan = []

path = "/Users/mehdiregina/FilRouge/Mehdi/data_talan"
filenames = sorted(glob(os.path.join(path,"*.txt")))
print(len(filenames))
for file in filenames:
    liste_cv_talan.append(open(file).read())

In [ ]:
liste_cv_talan[0]

In [ ]:
#suppression des saut de lignes
liste_cv_talan = [del_line_feed(text).lower() for text in liste_cv_talan]

In [ ]:
#suppression de la ponctuation
liste_cv_talan_no_punc = [del_punct(text) for text in liste_cv_talan]


In [ ]:
#selectionner seulement cvs fr
liste_cv_talan_fr = get_cv_langue(liste_cv_talan_no_punc,'french')

nb_cv = len(liste_cv_talan_no_punc)
nb_cv_fr = len(liste_cv_talan_fr)

print("proportion cv french :",1- ((nb_cv-nb_cv_fr)/nb_cv))

In [ ]:
#On supprime les caractères étranges, accents et stop words
liste_cv_treated_talan = [text_treatment(text) for text in liste_cv_talan_fr]

In [ ]:
#remove stop word
liste_cv_talan_no_stop = [remove_stopwords(text,stop_words_main) for text in liste_cv_treated_talan]

In [ ]:
#facultatif add only for talan cv (delete numbers) -> could be use for the preprocessing in general !
liste_cv_talan_clean = [re.sub('[0-9 ]+', ' ', text) for text in liste_cv_talan_no_stop]
liste_cv_talan_clean[2]

In [ ]:
totalvocab_stemmed_talan = []
totalvocab_tokenized_talan = []
for text in liste_cv_talan_no_stop:
    allwords_stemmed = tokenize_and_stem(text) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed_talan.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    allwords_tokenized = tokenize_only(text)
    totalvocab_tokenized_talan.extend(allwords_tokenized)

vocab_frame_talan = pd.DataFrame({'words': totalvocab_tokenized_talan}, index = totalvocab_stemmed_talan)
print('there are ' + str(vocab_frame_talan.shape[0]) + ' items in vocab_frame')
vocab_frame_talan

In [ ]:
vocab_frame_talan.loc['obie']

#### ACP X KMEANS TALAN

In [ ]:
#TF IDF BOW Representation
tf_vect = TfidfVectorizer(stop_words=stop_words_main,max_df=0.8,min_df=0.1,\
                           preprocessor=text_treatment,tokenizer=tokenize_and_stem)
test_idf_talan = tf_vect.fit_transform(liste_cv_talan_clean)

vocab_liste_talan = tf_vect.get_feature_names()

In [ ]:
vocab_liste_talan

In [ ]:
#LSA
svd = TruncatedSVD(40)
normalizer = Normalizer(copy=False)
lsa_talan = make_pipeline(svd, normalizer)

test_idf_reduced_talan = lsa_talan.fit_transform(test_idf_talan)
print(test_idf_reduced_talan.shape)

#keep the same variance than for the the indeed data above
explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

In [ ]:
#kmeans
n_class = 5
km = KMeans(n_clusters=n_class, init='k-means++', max_iter=100, n_init=1)
km.fit(test_idf_reduced_talan)

In [ ]:
get_cluster_words(7,km.cluster_centers_,vocab_frame_talan,vocab_liste_talan)

#### NMF X KMEANS TALAN

In [ ]:
# on part d'un bag of words tf-idf
tf_vect_2 = TfidfVectorizer(stop_words=stop_words_main,max_df=0.8,min_df=0.05,\
                           preprocessor=text_treatment,tokenizer=tokenize_and_stem)

test_idf_2_talan = tf_vect_2.fit_transform(liste_cv_talan_clean)

vocab_liste_nmf_talan = tf_vect_2.get_feature_names()

In [ ]:
liste_topic, liste_cluster = get_kmens_cluster_words_nmf(test_idf_2_talan,20,5,5,vocab_frame_talan,vocab_liste_nmf_talan)

In [ ]:
liste_topic

In [ ]:
liste_cluster